In [ ]:
# Things to finish before reaching out to GroteRoutepaden:
# 1: ColorLine -> DONE
# 2: Make plots of highway/surface/tracktype
# 3: Determine paved status rules
# 4: Fix double pathing
# 5: Make plots: percentage paved vs. unpaved, length of individual paved/unpaved sections
# 6: Brainstorm about other outputs that may be useful: closest town, topography type (forest, plains, etc.), elevation profile

In [51]:
# Preamble
from IPython.display import IFrame
from itertools import groupby 
from csv import writer
import networkx as nx
import pandas as pd
import numpy as np
import osmnx as ox
import warnings
import folium
import branca.colormap as cm
%matplotlib inline

In [52]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [53]:
# --- Load and process the GPX track
trail = 'gr131'
points_per_batch = 100
filename_in = 'data/' + trail + '.csv'
hike = pd.read_csv(filename_in)
coords = hike[['latitude','longitude']].values.tolist()
n_batch = int(np.ceil(hike.shape[0]/points_per_batch))

In [54]:
filename = 'data/' + trail + '_0to' + str(points_per_batch) + '.csv'
df = pd.read_csv(filename)

In [55]:
# Loading data and merging
for b in range(1,n_batch): # b is the batch counter
#     print(b)
    n1 = b*points_per_batch # First point
    n2 = min(n1 + points_per_batch, len(hike)) # Last point
    filename = 'data/' + trail + '_' + str(n1) + 'to' + str(n2) + '.csv'
    df_new = pd.read_csv(filename)
    df = pd.concat([df,df_new],ignore_index=True)

## Plotting GPX track versus OSM route

In [71]:
# Trying out ColorLine
df['paved'] = df['highway'].isin(['track','path'])
x = df['x0'].values.tolist()
y = df['y0'].values.tolist()
x.extend(df.tail(1)['x1'])
y.extend(df.tail(1)['y1'])
colors = df['paved'].values.tolist()
colors[colors==True] = 1
colors[colors==False] = 0
xy0 = list(zip(x,y))
xy = [[coord[0],coord[1]] for coord in xy0]

colormap = cm.LinearColormap(colors=['black','#239621'],vmin=0.25,vmax=0.75,index=[0.25,0.75])

In [72]:
mid = int(len(coords)/2)
chart = folium.Map(location=coords[mid], zoom_start=12, tiles="OpenStreetMap") # Map setup
    
newline = folium.ColorLine(positions=xy, colors=colors, colormap=colormap, weight=3)
newline.add_to(chart)
    
newline = folium.PolyLine(locations=coords, color='red', weight=1, popup='None')
newline.add_to(chart)

# Render the map
filepath = "data/chart.html"
chart.save(filepath)
IFrame(filepath, width=1000, height=500)

In [73]:
# mask_trackpath = df['highway'].isin(['track','path'])
# tot_unpaved = df[mask_trackpath]['d_cart'].sum()
# tot_paved = df[~mask_trackpath]['d_cart'].sum()
# tot = tot_unpaved + tot_paved
# print(f'Percentage paved: {tot_paved/tot}')
# print(f'Percentage unpaved: {tot_unpaved/tot}')
# df['tracktype'].value_counts()